# Week 4 Report
## A friend opening a resturant in Vancouver, BC, Cananda

**Tengkai Yu**

## Part 1

### Backgroud
**Assume one of my friends wants to open a restaurant in Vancouver, BC, Canada.  
Vancouver is one of the largest cities in Canada and holds a variety of populations.  
My friend is okay with any restaurant to open, but wishes for the best feedback from customers.**

### Why the analysis is important?
**Due to the variety of the population in Vancouver, any restaurant can find its customers.  
However, this variety also implies that any restaurant can face competition from other restaurants.  
Two main factors are affecting the restaurant feedback: the location and the type of dishes.  
For example, an Asain style restaurant in Chinatown might have more possible costumers but will face higher competition.  
If we do not handle these two variables carefully, we cannot maximize our feedback, or even face the survival difficulty.**

### Target audience
**So, who will need the analysis from this project?  
New restaurant owners, people who want to open their restaurants in any city, may or may not be Vancouver. They can be the same kind of people as my friend, a person who can open any restaurant anywhere. Or they might have a target location or target restaurant type.  
The other target audiences are those current restaurant owners who might be interested in evaluating their environment or moving their restaurant to a new location.  
The third type of audience is people who want to start their business in any place. They cannot use the result of our project directly but can use the model combined with their data collection. This combination can help them to evaluate their feedback in their area of business.**

## Part 2

### High level description
**We are going to retrieve the data from Foursquare API.  
In the previous assignments, we saw how to use the API to collect venue information, and we only need the restaurant information this time.  
The Foursquare API will provide the name, location, and rating.  
We are going to run the clustering method to separate those restaurants into different areas, then use the knn method to predict the expected rating.**

## Example data

In [60]:
import folium
# given that Vancouver's geo center's latitude and longitude are
latitude = 49.2827
longitude = -123.1207
map_vancouver = folium.Map(location=[latitude, longitude], zoom_start=14)
map_vancouver

In [61]:
CLIENT_ID = "4L44GMBDZZGMC4R4DYGQTNBW0EK2M5SKE5ILD5FF3GZL2HF4" # your Foursquare ID
CLIENT_SECRET = "NT1NK1NQYG2PQHKBGPXA4I5Z4JZAGB3ZSQ1JHTJ4CQVYV2ML" # your Foursquare Secret
VERSION = "20200419" # Foursquare API version

In [62]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [63]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9c387cbae9a2001b2ddd1e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Vancouver',
  'headerFullLocation': 'Downtown Vancouver, Vancouver',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 248,
  'suggestedBounds': {'ne': {'lat': 49.300700018000015,
    'lng': -123.09315798194572},
   'sw': {'lat': 49.26469998199998, 'lng': -123.14824201805428}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d524496529dcbff4ab3e4c4',
       'name': 'Hubbub Sandwiches',
       'location': {'address': '859 Hornby St',
        'crossStreet': 'btwn Robson & Smithe',
        'lat': 49.282078,
        'lng': -123.122

In [64]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [65]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
nearby_venues.head(10)

(100, 4)


<ipython-input-65-6de2aa623136>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Hubbub Sandwiches,Sandwich Place,49.282078,-123.122870
1,Le Crocodile Restaurant,French Restaurant,49.282658,-123.125287
2,Gotham Steakhouse & Cocktail Bar,Steakhouse,49.282830,-123.115865
3,Medina Café,Breakfast Spot,49.280565,-123.116859
4,The Keg Steakhouse + Bar - Dunsmuir,Restaurant,49.283438,-123.116363
5,Joe Fortes Seafood & Chop House,Seafood Restaurant,49.285017,-123.124410
6,49th Parallel & Lucky's Doughnuts,Donut Shop,49.285908,-123.122886
7,Shizen Ya,Japanese Restaurant,49.280880,-123.124552
8,Hy's Steakhouse,Steakhouse,49.284676,-123.119163
9,Gyu-Kaku Japanese BBQ,Japanese Restaurant,49.280277,-123.124838


In [66]:
# now try to plot these restaurant over the map, we have
# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver) 
map_vancouver

**The search mathod can be only called once a day so I will use them in week 5 notebook**